# 🎶 Predicting Lyrics
<hr>
<br>
<b> Group name: <strike>AA</strike> <i>My Machine Learning Romance</i> </b> <br>
- Fiorella Tenorio <br>
- Katja Wittfoth <br>
- Rebecca Reilly <br>
- Victoria Suarez <br>
- Viviana Márquez <br>

<br>
<img src='USF.png', style="width:200px;" align="left">
Friday, December 7th, 2018. <br>
<b> MSDS 621 </b> - Introduction to Machine Learning 

## Goal 
<center>
    <img src="are you smarter.png">
    <big>Predict genre based on lyrics</big><br><br>
    🥊 Supervised <i>vs.</i> Unspervised  <i>vs.</i> Humans 🥊
</center> 

## Workflow Pipeline
<center>
    <img src="workflow2.png" style="height:500px">
</center> 

In [1]:
# Libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction import stop_words
from sklearn.model_selection import GridSearchCV


import glob
import os
import pandas as pd
import matplotlib.pyplot as plt   
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mlxtend.classifier import EnsembleVoteClassifier

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score


import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
df = pd.read_csv('lyrics.csv')

In [3]:
# Clean data
def clean_lyrics(text):
    text = re.sub('\n', ' ', text)  # removes new lines
    text = re.sub('\d', '', text)  # removes numbers
    text = re.sub('\t', '', text)  # removes tabs
    words = text.split(" ")
    words = [w for w in words if len(w) > 2]  # removes a, an, to, at, be, ...
    return ' '.join(words)


df = df[pd.notnull(df['lyrics'])]
df = df[df['genre'] != 'Not Available']
df = df[df['genre'] != 'Other']
df['lyrics_clean'] = df.lyrics.apply(clean_lyrics)

df_clean = df[(df.genre == 'Country') | (df.genre == 'Metal') |
              (df.genre == 'Hip-Hop') | (df.genre == 'Jazz') | (df.genre == 'Electronic')]

df_clean = df_clean.sample(frac=1)  # shuffle

## 📊 1. Get data and pre-process

- 380,000+ lyrics  from MetroLyrics in <a href="https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics">Kaggle</a>

#### 🛁 Cleaning 
- Remove: `Null` lyrics, "Not available" genre, punctuation, numbers, symbols, spaces, stop words.
- Shuffle data.
- **Genres**: Country, Metal, Hip-Hop, Jazz, Electronic.

In [4]:
df_clean.head()

,index,song,year,artist,genre,lyrics,lyrics_clean
80497,80497,angst-atmet-mord,2007,bethlehem,Metal,"[This is a cool song, too bad it's only 3:23 a...","[This cool song, too bad it's only and instrum..."
104484,104484,eiskalt,2009,culcha-candela,Electronic,"T, t, tau mich auf\nAh, ah eiskalt, eiskalt, e...","tau mich auf Ah, eiskalt, eiskalt, eiskalt, ei..."
185957,185957,blown-away,2006,bizzy-bone,Hip-Hop,"[Intro: Bizzy (Playalitical)]\nWhat's goin on,...","[Intro: Bizzy (Playalitical)] What's goin on, ..."
220728,220728,sube,2010,ana-tijoux,Hip-Hop,"Ho! Oouuh! What?\nEn sus marcas, apuntar las i...","Ho! Oouuh! What? sus marcas, apuntar las ideas..."
742,742,like-mich-am-arsch,2015,deichkind,Hip-Hop,"Danke fÃ¼r den Kommentar, das gefÃ¤llt mir\nLi...","Danke fÃ¼r den Kommentar, das gefÃ¤llt mir Lik..."


## 👠 2. Modeling

### ✂️ Splitting data

In [5]:
train, test = train_test_split(df_clean, test_size=0.2)
train_lyrics = train['lyrics']
train_genre = train['genre']

### 🥊 Supervised model: Multinomial Naive-Bayes

In [6]:
train_lyrics = train['lyrics']
test_lyrics = test['lyrics']

In [11]:
# Pipelines
pipe_nb_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('nb', MultinomialNB())
])

pipe_nb_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('nb', MultinomialNB())
])

pipe_svc_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())
])

pipe_svc_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())
])

pipelines = [pipe_nb_cv, pipe_nb_tf, pipe_svc_cv, pipe_svc_tf]

In [12]:
# Fit pipe
for pipe in pipelines:
    pipe.fit(train_lyrics, train_genre)

In [13]:
methods = []

for pipe in pipelines:
    name = pipe.steps[-1][1].__class__.__name__.split('.')[-1][0:6]
    vect = pipe.steps[0][1].__class__.__name__.split('.')[-1][0:6]
    
    predicted = pipe.predict(test_lyrics)
    accuracy = accuracy_score(predicted,test['genre'])
    precision = precision_score(predicted,test['genre'])
    recall = recall_score(predicted, test['genre'])
    f1 = f1_score(predicted, test['genre'])
    conf_mat = confusion_matrix(test['genre'], predicted)
    
    methods.append([name,vect,accuracy,precision,recall,f1,conf_mat])

methods=pd.DataFrame(methods)
methods.columns = ['Model', 'Vectorizer', 'Accuracy', 'Precision', 'Recall', 'F1', "Confussion_Matrix"]
methods["Strategy"] = methods["Model"]+ " -- " + methods["Vectorizer"]
methods.set_index(["Strategy"],inplace=True)

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [ ]:
grid_params = dict(clf__alpha = np.arange(start=0, stop = 0.25, step = 0.05))

gs = GridSearchCV(estimator=pipeline,  
                  param_grid=grid_params,
                  scoring='accuracy',
                  cv=10)

gs.fit(train_lyrics, train_genre)

In [ ]:
gs.best_params_

In [ ]:
gs.best_estimator_.steps[0][1]  # best model

In [ ]:
pred = gs.best_estimator_.predict(test_lyrics)

# Recap ✍️